# 14PD - semestrální práce
## První část analýza dat
### Vilém Pecen
Tento dokument představuje první část (analýza dat) semestrální práce z předmětu 14PD - Práce s daty.  
Pro vypracování bylo použito prostředí [JupyterLab](https://jupyter.org/), které interaktivně vizualizuje výstupy skriptů tvořených v jazyce [Python](https://www.python.org/).  
Téma této části semestrální práce je inspirováno činnostmi, které autor plní na stáži u [Správy železnic, s.o.](https://www.spravazeleznic.cz/web/guest) na Oddělení ETCS a moderních technologií Odboru zabezpečovací a telekomunikační techniky (O14). Jde o analýzu logu radioblokové centrály (RBC) systému jednotného evropského vlakového zabezpečovače ETCS.

## Formát vstupních dat
Následující odstavce popisují, jakým způsobem byla získána data pro analýzu.
### Log RBC
Záznamy o činnosti radioblokové centrály byly staženy v formátu `.xlsx` z diagnostického rozhraní zabezpečovacího zařízení DRA GDS dodavatele tohoto zařízení. Ačkoliv v samotném diagnostickém zobrazení je k dispozici více informací, exportované soubory obsahují pouze hlášení připojení vlaků k jednotlivým RBC a hlášení změny módu palubní jednotky - vždy opatřená časovou značkou události. Ostatní atributy záznamů nejsou pro účely této analýzy příliš významnné.  
Níže je uveden technický popis načtených dat, statistické charakteristiky numerických veličin a vzorek dat.

In [74]:
from IPython.display import display, Markdown
from ipywidgets import Output
import pandas as pd
from log import load_data
from column_names import *
example_data: pd.DataFrame = load_data("./data/20221221_praha.xlsx")
o = Output()
with o:
    example_data.info()
display(
    Markdown("### Vzorek dat:"), example_data.head(),
    Markdown("### Technické vlastnosti:"), o,
    Markdown("### Statistické parametry:"), example_data.describe()
)

### Vzorek dat:

,Čas,Název DLS,Událost,OBU ETCS ID,Číslo vlaku,Délka [m],Max. rychlost [km/h],Nápravový tlak [t],Povel stůj,Druh trakce,Druh vlaku
0,2022-12-16 12:28:39,ETCS Beroun,RBC 62 Beroun - 1022: vznik vlaku,1022,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN
1,2022-12-16 12:28:44,ETCS Beroun,"RBC 62 Beroun - 1022: vlak 55246, mód UN",1022,55246,30,110,16,NaN,NaN,NaN
2,2022-12-16 12:29:33,ETCS Beroun,"RBC 62 Beroun - 1022: vlak 55246, mód FS",1022,55246,30,110,16,NaN,NaN,NaN
3,2022-12-16 12:33:21,ETCS Beroun,RBC 62 Beroun - 1022: zánik vlaku,1022,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN
4,2022-12-16 13:39:51,ETCS Zdice - Plzeň,RBC 64 Ejpovice - Plzeň - 1022: vznik vlaku 55246,1022,55246,30,110,16,NaN,NaN,NaN


### Technické vlastnosti:

Output()

### Statistické parametry:

,OBU ETCS ID,Číslo vlaku,Délka [m],Max. rychlost [km/h],Nápravový tlak [t]
count,473.000000,323.0,274.0,274.0,274.0
mean,37142.162791,6284023.532508,83.755474,102.116788,21.678832
std,38273.538007,24285310.417489,167.106058,11.220772,1.25172
min,989.000000,43304.0,17.0,80.0,16.0
25%,13001.000000,52529.0,20.0,100.0,22.0
50%,13001.000000,53114.0,20.0,100.0,22.0
75%,94144.000000,55246.0,20.0,100.0,22.0
max,94360.000000,100663296.0,680.0,160.0,22.0


### Data o poloze železničních stanic
Pro účely vizualizace se autor rozhodl využít prostorová data o poloze železničních stanic v ČR dostupná z projektu [OpenStreetMap](https://www.openstreetmap.org/#map=8/49.611/15.408). Pomocí nástroje [Overpass](https://overpass-turbo.eu/) byl vytvořen "dotaz" (viz soubor geo/query.txt), pomocí nějž je možné z mapových dat vyfiltrovat pouze objekty s atributy popisujícími železniční stanici. (Určení prostorových limitů pro dotaz pak představuje viditelná část mapy při spuštění vyhledávání.) Data byla stažena ve formátu `.geojson`, pro účely využití k vizualizaci byly s cílem zrychlení opakovaného načítání uloženy na lokální úložiště ve formátu `.parquet` ([Apache](https://parquet.apache.org/)).  
Jako podkladová mapa bude použita [OpenRailwayMap](https://www.openrailwaymap.org/).  
Pozice stanic slouží k přibližnému vyznačení oblasti, z níž jsou v daném načteném `.xlsx` souboru s logy dostupná data. K tomutu určení jsou využity "názvy" jednotlivých RBC, které často odpovídají označení některé železniční stanice. Alternativní postup hledání geografických umístění pro tyto názvy pomocí tzv. [geokódování](https://en.wikipedia.org/wiki/Address_geocoding) se ukázal jako poměrně náročnější a méně přesný.

### Cíl analýzy
Cílem analýzy souborů logu je nalezení nenahlášených provozních incidentů a výpadků systému ETCS. Zatím byly zpracovány postupy hledání nouzových zastavení vlaků systémem ETCS (přechod do tzv. módu Trip - TR), výpadek spojení mezi palubní a traťovou částí a neoprávněná aktivace procedury "Potlačení - Override" strojvedoucím.
- **Mód Trip** je detekován prostým výskytem kódu `TR` v daném záznamu
- **Rozpad spojení** je vyhodnocován při hlášení odpojení vlaku jednoucího v módu FS (Plný dohled - Full Supervision) od RBC, pokud poté následuje opětovné připojení ke stejnému RBC v módu TR, PT (Post Trip), či SR (Staff Responsible) \[Při rozpadu spojení dojde také k nouzovému zastevení vlaku, systém o této události ale nemusí dostat hlášení\.]
- **Nedovolené Potlačení** je vyhodnoceno v případě, kdy vlak přejde z módu FS přes mód SR do módu OS (On Sight), kdy mezi těmito přechody je (s určitou tolerancí) časový odstup 100, nebo 60 s - podle verze systému.

## Popis řešení
Při vypracování této práce byly použity následující rozšiřující balíčky pro Python:
- **Pandas, Numpy** - práce s daty
- **GeoPandas, ipyleaflet** - vykreslování map, práce s geografickými daty
- **Jupyter Widgets (ipywidgets)** - interaktivní ovládací prvky v prostředí JupyterLab
- **Matplotlib** - tvorba grafů
Seznam veškerých instalovaných balíčků je exportován do souboru `requirements.txt`.
#### Struktuta
Hlavní soubor, který slouží k zobrazení výstupu a ovládání analýzy, je `main.ipynb` (soubor aplikace JupyterLab).  
Část kódu vykreslující a obsluhující uživatelské rozhraní je v souboru `ui.py` a `map_tools.py`.  
Kód provádějící samotnou datovou analýzu (načítání, zpracování a uložení připravení výsledků pro zobrazení) je pak vyčleněn do souboru `log.py`.  
Ostatní soubory slouží pro definici konstant.  

Ve složce `data` jsou uloženy zdrojové soubory ve formátu `.xlsx`, složka `geo` obsahuje stažená data o poloze železničních stanic.  

Následující sekce již obsahují ovládací rozhraní a výstupy pro analýzu jednotlivých vstupních souborů `.xlsx`.

In [61]:
# JupyterLab rendering
from IPython.display import display, clear_output, Markdown

import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_columns', 0)

import ipywidgets as widgets
from ipyfilechooser import FileChooser

import os

# re-import when running import statements (load changes in imported file)
%load_ext autoreload
%autoreload 2

%matplotlib widget

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
# define constants
from column_names import *
from modes import *
from errors import *
from parameters import *

In [63]:
from log import analyse
from ui import render_ui

In [76]:
file_chooser = FileChooser(
    "./data",
    filter_pattern="*xlsx",
    change_desc="Změnit",
    select_desc="Vybrat"
)

output = widgets.Output()
  
def update(_):
    file_name = file_chooser.selected_filename
    with output:
        clear_output()
        print("Working...")
        data=analyse(os.path.join("data", file_name))
        clear_output()
        display(render_ui(file_name, data))

refresh_button = widgets.Button(
    description='Načíst',
    tooltip='Smazat zobrazené výsledky a načíst ze souboru',
    icon='refresh'
)
refresh_button.on_click(update)

file_selection = widgets.HBox([
    file_chooser,
    refresh_button
])

display(
    Markdown("## Analýza diagnostiky RBC"),
    Markdown("#### Vyber soubor"),
    file_selection,
    output
)

## Analýza diagnostiky RBC

#### Vyber soubor

Output()